In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Parameter yang akan diuji
param_grid = {
    'C': [1, 10, 100],                        # regularisasi ringan, sedang, kuat
    'epsilon': [0.01, 0.1, 0.5],              # toleransi kesalahan kecil hingga sedang
    'kernel': ['linear', 'rbf', 'poly'],      # core kernel yang paling umum digunakan
    'gamma': ['scale', 0.01, 0.1],            # cukup untuk variasi halus
    'degree': [2, 3],                         # cukup untuk polynomial order rendah
    'coef0': [0.0, 0.5]                       # nilai rendah dan sedang untuk poly/sigmoid
}

# Dapatkan kombinasi param yang relevan per kernel
filtered_param_grid = generate_kernel_specific_param_grid(param_grid)

# Inisialisasi model SVR
svr_model_name = results_df.iloc[0]['Model']
svr_model = models[svr_model_name]

# Inisialisasi objek GridSearchCV
grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)

# Lakukan pencarian parameter terbaik
grid_search.fit(X_train, y_train)

In [ ]:
print("Best params:", grid_search.best_params_)

In [ ]:
# Ambil parameter terbaik kemudian train ulang model
best_params = grid_search.best_params_
svr_grid_best = SVR(**best_params)
svr_grid_best.fit(X_train, y_train)

In [ ]:
# rmse untuk data train
y_train_pred = svr_grid_best.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# rmse untuk data test
y_test_pred = svr_grid_best.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("RMSE on training set:", rmse_train)
print("RMSE on testing set:", rmse_test)

# CatBoost Regressor

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# Splitting dataset menjadi train dan test dengan perbandingan 8:2  
rumah123_df = pd.read_csv("../../data/rumah123_ready.csv")
X = rumah123_df.drop(columns=['price'])
y = rumah123_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [29]:
param_grid = {
    'iterations': [2000],       # 1000, 1500, 2000
    'learning_rate': [0.15],    # 0.01, 0.1, 0.15
    'depth': [7],               # 6, 8, 10
    'l2_leaf_reg': [2],         # 1, 3, 5
    'random_strength': [0.9, 1, 2],     # 1, 5, 10
    'bagging_temperature': [0], # 0, 1, 5
    # 'border_count': [32, 64, 128],       # 32, 64, 128, 254
    'verbose': [0],
    'od_type': ['IncToDec'],
    'od_wait': [30, 50],
}

model = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    # logging_level='Silent', 
    thread_count=-1
)

# Tangkap hasil grid search
grid_result = model.grid_search(
    param_grid,
    X=Pool(X_train_scaled, y_train),
    cv=3,
    partition_random_seed=42,
    refit=True,
    calc_cv_statistics=True,
    verbose=3
)

# Ambil best_params
best_params = grid_result['params']
print("Best parameters found:", best_params)



bestTest = 266.8390972
bestIteration = 1088

0:	loss: 266.8390972	best: 266.8390972 (0)	total: 8.15s	remaining: 40.7s

bestTest = 266.8390972
bestIteration = 1088


bestTest = 265.5830483
bestIteration = 1397


bestTest = 265.5830483
bestIteration = 1397

3:	loss: 265.5830483	best: 265.5830483 (2)	total: 31.4s	remaining: 15.7s

bestTest = 267.7219215
bestIteration = 1705


bestTest = 267.7219215
bestIteration = 1705

5:	loss: 267.7219215	best: 265.5830483 (2)	total: 47.2s	remaining: 0us
Estimating final quality...
Training on fold [0/3]

bestTest = 278.1041632
bestIteration = 690

Training on fold [1/3]

bestTest = 273.5706015
bestIteration = 946

Training on fold [2/3]

bestTest = 272.7146384
bestIteration = 931

Best parameters found: {'random_strength': 1, 'od_wait': 30, 'iterations': 2000, 'verbose': 0, 'od_type': 'IncToDec', 'bagging_temperature': 0, 'l2_leaf_reg': 2, 'depth': 7, 'learning_rate': 0.15}
